In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By

In [2]:
from time import sleep
import random
import pandas as pd

In [3]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

In [4]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import requests

In [5]:
import yaml

with open('secrets.yaml', 'r') as file:
    secrets = yaml.safe_load(file)

my_id = secrets['my_id']
my_pw = secrets['my_pw']

In [6]:
# 바이너리 오류가 발생할 경우
# 크롬 바이너리 경로와 크롬 드라이버 바이너리 경로를 둘다 명시해줘야함
options = webdriver.ChromeOptions()
options.binary_location = "/Applications/Google Chrome.app/Contents/MacOS/Google Chrome"
chrome_driver_binary = "./chromedriver"
driver = webdriver.Chrome(chrome_driver_binary, chrome_options=options)
print("##########driver open##########")

/var/folders/rg/mxvp06m56_g09wl1jz84qkhh0000gn/T/ipykernel_92039/699548518.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_driver_binary, chrome_options=options)
/var/folders/rg/mxvp06m56_g09wl1jz84qkhh0000gn/T/ipykernel_92039/699548518.py:6: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_driver_binary, chrome_options=options)


##########driver open##########


In [7]:
wait = WebDriverWait(driver, 600)

In [8]:
site = "https://portal.hanyang.ac.kr/sso/lgin.do"

driver.get(url = site)
sleep(random.randint(1, 3))

# 로그인

In [9]:
# 수강신청 사이트와 포탈을 선택하는 창에서 포탈 선택
# sugang_select = driver.find_element(By.XPATH, '/html/body/div[4]/table/tbody/tr[1]/td/table/tbody/tr[1]/td[2]/img')
# sugang_select.click()


# id란에 id 입력
id_field = driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/form/div/fieldset/p[1]/input')
id_field.send_keys(my_id)

# pw란에 pw 입력
password_field = driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/form/div/fieldset/p[2]/input')
password_field.send_keys(my_pw)

# 로그인 버튼을 클릭
login_button = driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/form/div/fieldset/p[3]/a')
login_button.click()

# 비밀번호 다음에 변경
pw_change = driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div/div[3]/span[2]/input')
pw_change.click()

# 교육창 닫기
WebDriverWait(driver, 10).until(EC.alert_is_present())
alert = driver.switch_to.alert
alert.dismiss()

# 공지사항 게시판 들어가기
noti_button = driver.find_element(By.XPATH, '/html/body/div[3]/div[2]/div[2]/div[2]/div[2]/div[1]/div[1]/p/a')
noti_button.click()

In [10]:
data = {
        '번호': [],
        '분류': [],
        '등록시간': [],
        '작성자' : [],
        '제목': [],
        '본문': [],
        'url': [],
        '첨부파일': [],
       }

In [ ]:
# 10페이지 넘김 버튼때문에 만든 변수
page_check = 0

while True:
    
    try:
        # 첫 페이지만 span[3]/* 이고 나머지 페이지는 span[2]/* 이므로 예외처리
        try:
            wait = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div[2]/div[2]/div[2]/div[2]/form/div[2]/div[4]/span/span[3]/*")))
            page_num = driver.find_elements(By.XPATH, "/html/body/div[3]/div[2]/div[2]/div[2]/div[2]/form/div[2]/div[4]/span/span[3]/*")
        except:
            wait = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div[2]/div[2]/div[2]/div[2]/form/div[2]/div[4]/span/span[2]/*")))
            page_num = driver.find_elements(By.XPATH, "/html/body/div[3]/div[2]/div[2]/div[2]/div[2]/form/div[2]/div[4]/span/span[2]/*")
    # 10페이지 단위가 넘어가면 span[3]나 span[2]가 아니라 span으로 바뀜
    except: 
        wait = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div[2]/div[2]/div[2]/div[2]/form/div[2]/div[4]/span/span/*")))
        page_num = driver.find_elements(By.XPATH, "/html/body/div[3]/div[2]/div[2]/div[2]/div[2]/form/div[2]/div[4]/span/span/*")
    
    for j in range(len(page_num)):
        
        # 페이지 로드 대기
        sleep(random.randint(1, 3))
        
        # stale 오류때문에 변수 다시 정의
        try:
            try:
                wait = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div[2]/div[2]/div[2]/div[2]/form/div[2]/div[4]/span/span[3]/*")))
                page_num = driver.find_elements(By.XPATH, "/html/body/div[3]/div[2]/div[2]/div[2]/div[2]/form/div[2]/div[4]/span/span[3]/*")
            except:
                wait = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div[2]/div[2]/div[2]/div[2]/form/div[2]/div[4]/span/span[2]/*")))
                page_num = driver.find_elements(By.XPATH, "/html/body/div[3]/div[2]/div[2]/div[2]/div[2]/form/div[2]/div[4]/span/span[2]/*")
        except:
            wait = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div[2]/div[2]/div[2]/div[2]/form/div[2]/div[4]/span/span/*")))
            page_num = driver.find_elements(By.XPATH, "/html/body/div[3]/div[2]/div[2]/div[2]/div[2]/form/div[2]/div[4]/span/span/*")
        
        # 현재 페이지의 경우 클릭이 안되기 때문에 예외처리
        try:
            page_num[j].click()
        except:
            pass
        
        print('****************************************\n')
        print(page_num[j].text + '번째 페이지 크롤링 시작')
    
        # 마지막 페이지의 콘텐츠 수가 다를 수 있으므로
        wait = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div[2]/div[2]/div[2]/div[2]/form/div[2]/div[2]/div/table/tbody/tr")))
        content_num = driver.find_elements(By.XPATH, "/html/body/div[3]/div[2]/div[2]/div[2]/div[2]/form/div[2]/div[2]/div/table/tbody/tr")

        for i in range(len(content_num)):

            # 페이지 로드 대기
            sleep(random.randint(1, 3))

            print("--------------------------------------------")

            # 캠퍼스(번호)
            wait = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div[2]/div[2]/div[2]/div[2]/form/div[2]/div[2]/div/table/tbody/tr[" + str(i+1) + "]/td[1]/span[1]")))
            campus = driver.find_elements(By.XPATH, "/html/body/div[3]/div[2]/div[2]/div[2]/div[2]/form/div[2]/div[2]/div/table/tbody/tr[" + str(i+1) + "]/td[1]/span[1]")[0]
            campus = campus.text
            if campus == 'H':
                data['번호'].append('전체')
                print('번호 : 전체')
            elif campus == 'S':
                data['번호'].append('서울')
                print('번호 : 서울')
            else:
                data['번호'].append('에리카')
                print('번호 : 에리카')

            # 분류
            wait = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div[2]/div[2]/div[2]/div[2]/form/div[2]/div[2]/div/table/tbody/tr[" + str(i+1) + "]/td[1]/span[2]")))
            cate = driver.find_elements(By.XPATH, "/html/body/div[3]/div[2]/div[2]/div[2]/div[2]/form/div[2]/div[2]/div/table/tbody/tr[" + str(i+1) + "]/td[1]/span[2]")[0]
            cate = cate.text
            data['분류'].append(cate)
            print('분류 :', cate)

            # 등록시간 수집
            wait = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div[2]/div[2]/div[2]/div[2]/form/div[2]/div[2]/div/table/tbody/tr[" + str(i+1) + "]/td[3]")))
            upload_time = driver.find_elements(By.XPATH, "/html/body/div[3]/div[2]/div[2]/div[2]/div[2]/form/div[2]/div[2]/div/table/tbody/tr[" + str(i+1) + "]/td[3]")[0]
            upload_time = upload_time.text
            # 2023년 이전 자료는 크롤링 하지 않음
            if int(upload_time[:4]) < 2023:
                break
            data['등록시간'].append(upload_time)
            print('등록시간 :', upload_time)

            # 작성자 수집
            wait = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div[2]/div[2]/div[2]/div[2]/form/div[2]/div[2]/div/table/tbody/tr[" + str(i+1) + "]/td[2]")))
            writer = driver.find_elements(By.XPATH, "/html/body/div[3]/div[2]/div[2]/div[2]/div[2]/form/div[2]/div[2]/div/table/tbody/tr[" + str(i+1) + "]/td[2]")[0]
            writer = writer.text
            data['작성자'].append(writer)
            print('작성자 :', writer)

            # 제목 수집
            wait = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div[2]/div[2]/div[2]/div[2]/form/div[2]/div[2]/div/table/tbody/tr[" + str(i+1) + "]/td[1]/span[3]")))
            title = driver.find_elements(By.XPATH, "/html/body/div[3]/div[2]/div[2]/div[2]/div[2]/form/div[2]/div[2]/div/table/tbody/tr[" + str(i+1) + "]/td[1]/span[3]")[0]
            title = title.text
            data['제목'].append(title)
            print('제목 :', title)

            # 글 클릭
            # click intercepted error -> retry
            try:
                wait = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[3]/div[2]/div[2]/div[2]/div[2]/form/div[2]/div[2]/div/table/tbody/tr[" + str(i+1) + "]")))
                # click intercepted error -> sleep 
                sleep(random.randint(1, 3))
                con_button = driver.find_elements(By.XPATH, "/html/body/div[3]/div[2]/div[2]/div[2]/div[2]/form/div[2]/div[2]/div/table/tbody/tr[" + str(i+1) + "]")[0]
                con_button.click()
            except:
                wait = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[3]/div[2]/div[2]/div[2]/div[2]/form/div[2]/div[2]/div/table/tbody/tr[" + str(i+1) + "]")))
                # click intercepted error -> sleep 
                sleep(random.randint(1, 3))
                con_button = driver.find_elements(By.XPATH, "/html/body/div[3]/div[2]/div[2]/div[2]/div[2]/form/div[2]/div[2]/div/table/tbody/tr[" + str(i+1) + "]")[0]
                con_button.click()

            # 본문 수집
            wait = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div[2]/div[2]/div[2]/div[2]/table/tbody/tr[4]/td")))
            # wait를 앞에서 걸어도 로드가 덜 되어서 수집이 안되는 경우 때문에 sleep 걸었음
            sleep(random.randint(1, 3))
            main = driver.find_elements(By.XPATH, "/html/body/div[3]/div[2]/div[2]/div[2]/div[2]/table/tbody/tr[4]/td")[0]
            main = main.get_attribute('innerHTML')
            data['본문'].append(main)
            print('본문 :', main[:50], '~')

            # url 수집
            data['url'].append(driver.current_url)
            print('url :',driver.current_url)

            # 첨부파일 수집
            try:
                # 현재 한양인의 경우 첨부파일이 html 코드로 노출되어있지 않고, javascript로 동적으로 작동하는 것 같음
                # 일단 넘어가고 이에대한 해결방법 찾아야함
                empty_list = []
                data['첨부파일'].append(empty_list)
                print('첨부파일 : 없음')

            except:
                empty_list = []
                data['첨부파일'].append(empty_list)
                print('첨부파일 : 없음')

            # 목록버튼 클릭
            # click intercepted error -> retry
            try:
                wait = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[3]/div[2]/div[2]/div[2]/div[2]/div[1]/span/input")))
                # click intercepted error -> sleep 
                sleep(random.randint(1, 3))
                list_button = driver.find_elements(By.XPATH, "/html/body/div[3]/div[2]/div[2]/div[2]/div[2]/div[1]/span/input")[0]
                list_button.click()
            except:
                wait = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[3]/div[2]/div[2]/div[2]/div[2]/div[1]/span/input")))
                # click intercepted error -> sleep 
                sleep(random.randint(1, 3))
                list_button = driver.find_elements(By.XPATH, "/html/body/div[3]/div[2]/div[2]/div[2]/div[2]/div[1]/span/input")[0]
                list_button.click()
            
        # 2023년 이전 자료는 크롤링 하지 않음
        if int(upload_time[:4]) < 2023:
            break
    
    # 10페이지 넘김 버튼이 첫 10페이지에서는 a[2]/~ 인데 다음부터는 a[3]라서 예외처리
    # 첫 페이지에서는 이전 10페이지 버튼이 활성화가 되어있지 않기 때문인 것 같음
    # click intercepted error -> retry
    try:
        if page_check == 0:
            # click intercepted error -> sleep 
            sleep(random.randint(1, 3))
            wait = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[3]/div[2]/div[2]/div[2]/div[2]/form/div[2]/div[4]/span/a[2]/span/img")))
            next_button = driver.find_elements(By.XPATH, "/html/body/div[3]/div[2]/div[2]/div[2]/div[2]/form/div[2]/div[4]/span/a[2]/span/img")[0]
            next_button.click()
            page_check = 1
        else:
            sleep(random.randint(1, 3))
            wait = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[3]/div[2]/div[2]/div[2]/div[2]/form/div[2]/div[4]/span/a[3]/span/img")))
            next_button = driver.find_elements(By.XPATH, "/html/body/div[3]/div[2]/div[2]/div[2]/div[2]/form/div[2]/div[4]/span/a[3]/span/img")[0]
            next_button.click()
            page_check = 1
    except:
        if page_check == 0:
            # click intercepted error -> sleep 
            sleep(random.randint(1, 3))
            wait = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[3]/div[2]/div[2]/div[2]/div[2]/form/div[2]/div[4]/span/a[2]/span/img")))
            next_button = driver.find_elements(By.XPATH, "/html/body/div[3]/div[2]/div[2]/div[2]/div[2]/form/div[2]/div[4]/span/a[2]/span/img")[0]
            next_button.click()
            page_check = 1
        else:
            sleep(random.randint(1, 3))
            wait = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[3]/div[2]/div[2]/div[2]/div[2]/form/div[2]/div[4]/span/a[3]/span/img")))
            next_button = driver.find_elements(By.XPATH, "/html/body/div[3]/div[2]/div[2]/div[2]/div[2]/form/div[2]/div[4]/span/a[3]/span/img")[0]
            next_button.click()
            page_check = 1
        
        
    
    # 2023년 이전 자료는 크롤링 하지 않음
    if int(upload_time[:4]) < 2023:
        data['번호'].pop()
        data['분류'].pop()
        break


****************************************

1번째 페이지 크롤링 시작
--------------------------------------------
번호 : 전체
분류 : 일반
등록시간 : 2023.08.17
작성자 : 학생지원팀 / 조윤아
제목 : [금융감독원] 제18회 금융공모전 참여 안내
본문 : 
<p class="0CxSpFirst" style="margin: 0.0cm;line-h ~
url : https://portal.hanyang.ac.kr/port.do#!UDMwODIwMCRAXiRAXmNvbW0vZ2pzaCRAXk0wMDYyNjMkQF7qs7Xsp4Dsgqztla0kQF5NMDAzNzgxJEBeMGJlMjk1OTM2MjY0MjlkZmMzZjFiNjE4MDQ1YmM4MTcyYjg2ODMyZGYwZDMzM2JjMGY1ZGI0NzE5OWI5MDI4YQ==
첨부파일 : 없음
--------------------------------------------
번호 : 전체
분류 : 일반
등록시간 : 2023.08.17
작성자 : 학생지원팀 / 조윤아
제목 : [한국여성과학기술인육성재단] SAP, SAP Labs와 함께하는 원데이 글로벌 멘토링
본문 : <p class="HStyle0" style="margin: 0.0pt 0.0px 0.0p ~
url : https://portal.hanyang.ac.kr/port.do#!UDMwODIwMCRAXiRAXmNvbW0vZ2pzaCRAXk0wMDYyNjMkQF7qs7Xsp4Dsgqztla0kQF5NMDAzNzgxJEBeMGJlMjk1OTM2MjY0MjlkZmMzZjFiNjE4MDQ1YmM4MTcyYjg2ODMyZGYwZDMzM2JjMGY1ZGI0NzE5OWI5MDI4YQ==
첨부파일 : 없음
--------------------------------------------
번호 : 서울
분류 : 취업
등록시간 : 2023.08.17
작성자 : 커리어개발팀 / 이현진
제목 : [

In [ ]:
df = pd.DataFrame(data)

df.tail(8)

In [ ]:
df.info()

In [ ]:
driver.close()

In [ ]:
df.to_csv('HYin.csv', index=False)